In [19]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [26]:
df = pd.read_csv('/content/adult.csv', sep=',')
len(df)

32561

In [27]:
df = df[(df.astype(str) != ' ?').all(axis=1)]
len(df)

30162

In [28]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [29]:
df['income_bi'] = df.apply(lambda row: 1 if '>50K'in row['income'] else 0, axis=1)
df = df.drop(['income','fnlwgt','capital-gain','capital-loss','native-country'], axis=1)
df.head()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,income_bi
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,0


In [30]:
df.tail()

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,income_bi
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,38,0
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,1
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,40,0
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,20,0
32560,52,Self-emp-inc,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,40,1


In [31]:
df = pd.get_dummies(df, columns=['workclass', 'education', 'marital-status', 'occupation',
                                 'relationship', 'race', 'sex'])
df.head()

,age,education-num,hours-per-week,income_bi,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male
0,39,13,40,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
1,50,13,13,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
2,38,9,40,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,53,7,40,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,28,13,40,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,1,0


In [32]:
df.iloc[[0]].to_csv('prediction.csv', sep=',', encoding='utf-8', index=False)

In [33]:
# shuffle rows
df = df.sample(frac=1)

# split training and testing data
d_train = df[:25000]
d_test = df[25000:]

d_train_att = d_train.drop(['income_bi'], axis=1)
d_train_gt50 = d_train['income_bi']

d_test_att = d_test.drop(['income_bi'], axis=1)
d_test_gt50 = d_test['income_bi']

d_att = df.drop(['income_bi'], axis=1)
d_gt50 = df['income_bi']

# number of income > 50K in whole dataset:
print("Income >50K: %d out of %d (%.2f%%)" % (np.sum(d_gt50), len(d_gt50), 100*float(np.sum(d_gt50)) / len(d_gt50)))

Income >50K: 7508 out of 30162 (24.89%)


In [34]:
# Fit a decision tree
t = tree.DecisionTreeClassifier(criterion='entropy', max_depth=7)
t = t.fit(d_train_att, d_train_gt50)

In [35]:
t.score(d_test_att, d_test_gt50)

0.8244866330879504

In [36]:
scores = cross_val_score(t, d_att, d_gt50, cv=5)
# Show avarage score and +/- two standard deviations away (covering 95% or scores)
print('Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std()*2))

Accuracy: 0.83 (+/- 0.01)


In [37]:
sample_df = pd.read_csv('prediction.csv', sep=',')
sample_df = sample_df.drop(['income_bi'], axis=1)

In [38]:
predict_value = sample_df.iloc[0]
y_predict = t.predict([predict_value.tolist()])
y_predict[0]

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


0